In [1]:
import os,datetime,re
from utils import News, Site

In [ ]:
### 薬学部・薬学研究科 ###
class GLCNews(News):
    # this should be overrided
    # because the format of news will be different from the others
    def summary(self):
        time = re.search("\d+/\d+", self.tag.text).group()
        contents = self.tag.text.split(time)[-1].split()[-1]
        href = self.tag.get("href")
        self.content = f"《{time}》\n{contents}\n{href}"
        self.time = self.timeobj(time)

    def timeobj(self, timestr=""):
        year = "2020/"
        tmp = datetime.datetime.strptime(year + timestr, "%Y/%m/%d")
        return datetime.date(tmp.year, tmp.month, tmp.day)


class GLC(Site):
    path = os.path.join("..", os.path.join("sites_db", "pharm.pickle"))
    url = "https://www.insc.tohoku.ac.jp/english/news/othernews/"
    majors = ["english", "english"]

    def get(self):
        soup = self.request()
        # 以降、サイトに合わせて書き直す必要あり
        info_list = soup.find("div", class_="contents_wrap_box").find_all("a")
        info_list = self.abstract(info_list)
        # 固定情報
        stick1 = PharmNews(info_list[0])
        stick1_url = "https://www.tohoku.ac.jp/japanese/2020/04/news20200417-00.html"
        stick1.time = stick1.timeobj(timestr="4/17")
        stick1.content = f"《4/17》\n学生の入構制限について\n{stick1_url}"
        ###
        info_list = [PharmNews(info) for info in info_list]
        info_list.append(stick1)
        return self.dic(info_list)

    def abstract(self, a_tags):
        info_list = []
        for tag in a_tags:
            if tag.text not in  ["こちら", "東北大学新型コロナウイルスBCP対応ガイド", "新型コロナウイルス感染の疑いのある症状が出た場合の連絡",
                                "https://www.tohoku.ac.jp/japanese/2020/04/news20200417-00.html"]:
                href = tag.get("href")
                if (href[0:4] == "http") and (href.split("/")[2] != "www.tohoku.ac.jp"):
                    info_list.append(tag)
        return info_list
